In [1]:
import import_ipynb #Allows you to import ipynb files (TEMPORARY?)
from queue import PriorityQueue
import HybridAstar as has

importing Jupyter notebook from HybridAstar.ipynb
importing Jupyter notebook from Node.ipynb


In [2]:
maze = [[0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0]]
graph = [[0, 1, 0, 0, 0, 0],
         [1, 0, 1, 0, 1, 0],
         [0, 1, 0, 0, 0, 1],
         [0, 0, 0, 0, 1, 0],
         [0, 1, 0, 1, 0, 0],
         [0, 0, 1, 0, 0, 0]]
start = (0, 0)
end = (5, 5)
pf = has.HybridAStar(maze, start, end)
path = pf.GeneratePath()
print(path)

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 3), (5, 4), (5, 5)]
